In [1]:
! pip install torch numpy timm==0.5.4 tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
!git clone https://github.com/tsungchiehchen/Vision-Transformer.git

Cloning into 'Vision-Transformer'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 20 (delta 8), reused 18 (delta 6), pack-reused 0
Receiving objects: 100% (20/20), 17.05 KiB | 1.71 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [3]:
%cd ./Vision-Transformer

/content/Vision-Transformer


In [6]:
import argparse
import datetime
import os
import sys
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn

from timm.models import create_model

from engine import train_one_epoch, train_one_epoch_distillation, evaluate
from utils import get_training_dataloader, get_test_dataloader
import models

In [7]:
MEAN = (0.5070751592371323, 0.48654887331495095, 0.4409178433670343)
STD = (0.2673342858792401, 0.2564384629170883, 0.27615047132568404)
CHECKPOINT_PATH = './checkpoint'
MODEL_NAME = 'vit_base_patch16_224'
num_classes = 10
EPOCHS = 5
LR = 0.0001
WD = 0.0
shots = 1000

print(f"Creating model: {MODEL_NAME}")
model = create_model(
        MODEL_NAME,
        pretrained=False,
        num_classes=10,
        img_size=224)
device = 'cuda:0' # device = 'cpu'
model = model.to(device)

cifar10_training_loader = get_training_dataloader(
    MEAN,
    STD,
    num_workers=2,
    batch_size=16,
    shuffle=True,
    shots=shots
)

assert (shots*num_classes == len(cifar10_training_loader.dataset))

cifar10_test_loader = get_test_dataloader(
    MEAN,
    STD,
    num_workers=4,
    batch_size=256,
    shuffle=False
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WD)


n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('number of params:', n_parameters)

Creating model: vit_base_patch16_224


100%|██████████| 170498071/170498071 [00:02<00:00, 79445281.30it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
number of params: 86567656


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
print(f"Start training for {EPOCHS} epochs")

for epoch in range(1, EPOCHS+1):
    train_stats = train_one_epoch(
        model, criterion, cifar10_training_loader,
        optimizer, device, epoch)
    if epoch % 10 == 9:
        test_stats = evaluate(cifar10_test_loader, model, criterion, device)
        print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

test_stats = evaluate(cifar10_test_loader, model, criterion, device)
print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

Start training for 5 epochs
Epoch: [1]  [  0/625]  eta: 0:28:48  loss: 7.2056 (7.2056)  time: 2.7654  data: 0.4869  max mem: 2096
Epoch: [1]  [100/625]  eta: 0:01:44  loss: 2.2015 (2.4863)  time: 0.1713  data: 0.0057  max mem: 3065
Epoch: [1]  [200/625]  eta: 0:01:19  loss: 2.1340 (2.3034)  time: 0.1701  data: 0.0055  max mem: 3065
Epoch: [1]  [300/625]  eta: 0:00:59  loss: 2.0563 (2.2224)  time: 0.1778  data: 0.0084  max mem: 3065
Epoch: [1]  [400/625]  eta: 0:00:40  loss: 2.1087 (2.1713)  time: 0.1774  data: 0.0065  max mem: 3065
Epoch: [1]  [500/625]  eta: 0:00:22  loss: 1.8859 (2.1306)  time: 0.1793  data: 0.0054  max mem: 3065
Epoch: [1]  [600/625]  eta: 0:00:04  loss: 1.9450 (2.1032)  time: 0.1898  data: 0.0100  max mem: 3065
Epoch: [1]  [624/625]  eta: 0:00:00  loss: 2.0016 (2.0996)  time: 0.1834  data: 0.0051  max mem: 3065
Epoch: [1] Total time: 0:01:53 (0.1819 s / it)
Averaged stats: loss: 2.0016 (2.0996)
Epoch: [2]  [  0/625]  eta: 0:03:33  loss: 2.1157 (2.1157)  time: 0.341

In [9]:
# Calculate througput
start_time = time.time()
test_stats = evaluate(cifar10_test_loader, model, criterion, device)
end_time = time.time()
num_samples = len(cifar10_test_loader.dataset)
throughput = num_samples / (end_time - start_time)
print("Throughput: {}".format(throughput))

Test:  [ 0/40]  eta: 0:01:58  loss: 1.6862 (1.6862)  acc1: 40.6250 (40.6250)  acc5: 85.1562 (85.1562)  time: 2.9730  data: 1.8310  max mem: 3158
Test:  [20/40]  eta: 0:00:23  loss: 1.7490 (1.7590)  acc1: 36.3281 (35.9189)  acc5: 88.6719 (88.4301)  time: 1.0917  data: 0.0786  max mem: 3158
Test:  [39/40]  eta: 0:00:01  loss: 1.7465 (1.7480)  acc1: 36.7188 (36.1300)  acc5: 88.6719 (88.0300)  time: 1.0233  data: 0.0669  max mem: 3158
Test: Total time: 0:00:44 (1.1050 s / it)
* Acc@1 36.130 Acc@5 88.030 loss 1.748
Throughput: 226.21906627248507


# Q2 Fine-tuning Pretrained ViT

In [10]:
MEAN = (0.5070751592371323, 0.48654887331495095, 0.4409178433670343)
STD = (0.2673342858792401, 0.2564384629170883, 0.27615047132568404)
CHECKPOINT_PATH = './checkpoint'
MODEL_NAME = 'vit_base_patch16_224'
num_classes = 10
EPOCHS = 5
LR = 0.0001
WD = 0.0
shots = 1000

print(f"Creating model: {MODEL_NAME}")
model = create_model(
        MODEL_NAME,
        pretrained=True,
        num_classes=10,
        img_size=224)
device = 'cuda:0' # device = 'cpu'
model = model.to(device)

cifar10_training_loader = get_training_dataloader(
    MEAN,
    STD,
    num_workers=2,
    batch_size=16,
    shuffle=True,
    shots=shots
)

assert (shots*num_classes == len(cifar10_training_loader.dataset))

cifar10_test_loader = get_test_dataloader(
    MEAN,
    STD,
    num_workers=4,
    batch_size=256,
    shuffle=False
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WD)


n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('number of params:', n_parameters)

Creating model: vit_base_patch16_224


Downloading: "https://dl.fbaipublicfiles.com/deit/deit_base_patch16_224-b5f2ef4d.pth" to /root/.cache/torch/hub/checkpoints/deit_base_patch16_224-b5f2ef4d.pth
100%|██████████| 330M/330M [00:05<00:00, 59.9MB/s]


Files already downloaded and verified
Files already downloaded and verified
number of params: 85806346


In [11]:
print(f"Start training for {EPOCHS} epochs")

for epoch in range(1, EPOCHS+1):
    train_stats = train_one_epoch(
        model, criterion, cifar10_training_loader,
        optimizer, device, epoch)
    if epoch % 10 == 9:
        test_stats = evaluate(cifar10_test_loader, model, criterion, device)
        print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

test_stats = evaluate(cifar10_test_loader, model, criterion, device)
print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

Start training for 5 epochs
Epoch: [1]  [  0/625]  eta: 0:04:01  loss: 2.3790 (2.3790)  time: 0.3858  data: 0.1525  max mem: 3158
Epoch: [1]  [100/625]  eta: 0:01:38  loss: 2.0979 (2.2390)  time: 0.1886  data: 0.0055  max mem: 3158
Epoch: [1]  [200/625]  eta: 0:01:20  loss: 2.0896 (2.1396)  time: 0.1938  data: 0.0096  max mem: 3158
Epoch: [1]  [300/625]  eta: 0:01:01  loss: 1.8388 (2.0790)  time: 0.1859  data: 0.0053  max mem: 3158
Epoch: [1]  [400/625]  eta: 0:00:42  loss: 1.7095 (2.0115)  time: 0.1875  data: 0.0071  max mem: 3158
Epoch: [1]  [500/625]  eta: 0:00:23  loss: 1.5865 (1.9341)  time: 0.1861  data: 0.0057  max mem: 3158
Epoch: [1]  [600/625]  eta: 0:00:04  loss: 1.4219 (1.8547)  time: 0.1886  data: 0.0058  max mem: 3158
Epoch: [1]  [624/625]  eta: 0:00:00  loss: 1.4646 (1.8390)  time: 0.1900  data: 0.0065  max mem: 3158
Epoch: [1] Total time: 0:01:58 (0.1890 s / it)
Averaged stats: loss: 1.4646 (1.8390)
Epoch: [2]  [  0/625]  eta: 0:03:39  loss: 1.2923 (1.2923)  time: 0.351

In [12]:
# Calculate througput
start_time = time.time()
test_stats = evaluate(cifar10_test_loader, model, criterion, device)
end_time = time.time()
num_samples = len(cifar10_test_loader.dataset)
throughput = num_samples / (end_time - start_time)
print("Throughput: {}".format(throughput))

Test:  [ 0/40]  eta: 0:02:29  loss: 0.6622 (0.6622)  acc1: 77.7344 (77.7344)  acc5: 98.4375 (98.4375)  time: 3.7399  data: 2.5347  max mem: 3158
Test:  [20/40]  eta: 0:00:25  loss: 0.5921 (0.5938)  acc1: 80.0781 (80.3571)  acc5: 99.2188 (99.0699)  time: 1.1569  data: 0.1107  max mem: 3158
Test:  [39/40]  eta: 0:00:01  loss: 0.5921 (0.5866)  acc1: 80.4688 (80.3500)  acc5: 99.2188 (99.1700)  time: 1.0263  data: 0.0535  max mem: 3364
Test: Total time: 0:00:46 (1.1643 s / it)
* Acc@1 80.350 Acc@5 99.170 loss 0.587
Throughput: 214.69383078072903


# Q3 ViT model on a small device

In [ ]:
MEAN = (0.5070751592371323, 0.48654887331495095, 0.4409178433670343)
STD = (0.2673342858792401, 0.2564384629170883, 0.27615047132568404)
CHECKPOINT_PATH = './checkpoint'
MODEL_NAME = 'vit_tiny_patch16_224'
num_classes = 10
EPOCHS = 5
LR = 0.0001
WD = 0.0
shots = 1000

print(f"Creating model: {MODEL_NAME}")
model = create_model(
        MODEL_NAME,
        pretrained=True,
        num_classes=10,
        img_size=224)
device = 'cuda:0' # device = 'cpu'
model = model.to(device)

cifar10_training_loader = get_training_dataloader(
    MEAN,
    STD,
    num_workers=2,
    batch_size=16,
    shuffle=True,
    shots=shots
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WD)


n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('number of params:', n_parameters)

In [ ]:
print(f"Start training for {EPOCHS} epochs")

for epoch in range(1, EPOCHS+1):
    train_stats = train_one_epoch(
        model, criterion, cifar10_training_loader,
        optimizer, device, epoch)
    if epoch % 10 == 9:
        test_stats = evaluate(cifar10_test_loader, model, criterion, device)
        print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

test_stats = evaluate(cifar10_test_loader, model, criterion, device)
print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

In [ ]:
# Calculate througput
start_time = time.time()
test_stats = evaluate(cifar10_test_loader, model, criterion, device)
end_time = time.time()
num_samples = len(cifar10_test_loader.dataset)
throughput = num_samples / (end_time - start_time)
print("Throughput: {}".format(throughput))

# Q4 Knowledge Distillation

In [ ]:
# Step 1: Train the teacher

MODEL_NAME = 'vit_base_patch16_224'
num_classes = 10
EPOCHS = 5
LR = 0.0001
WD = 0.0

print(f"Creating model: {MODEL_NAME}")
teacher = create_model(
        MODEL_NAME,
        pretrained=True,
        num_classes=10,
        img_size=224)
device = 'cuda:0' # device = 'cpu'
teacher = teacher.to(device)



criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(teacher.parameters(), lr=LR, weight_decay=WD)


n_parameters = sum(p.numel() for p in teacher.parameters() if p.requires_grad)
print('number of params:', n_parameters)


In [ ]:
print(f"Start training for {EPOCHS} epochs")

for epoch in range(1, EPOCHS+1):
    train_stats = train_one_epoch(
        teacher, criterion, cifar10_training_loader,
        optimizer, device, epoch)
    if epoch % 10 == 9:
        test_stats = evaluate(cifar10_test_loader, teacher, criterion, device)
        print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

test_stats = evaluate(cifar10_test_loader, teacher, criterion, device)
print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")


In [ ]:
# save finetuned teacher model
torch.save(teacher.state_dict(), './teacher.pth')

In [ ]:

teacher = create_model(
        'vit_base_patch16_224',
        pretrained=True,
        num_classes=10,
        img_size=224)
device = 'cuda:0' # device = 'cpu'
teacher = teacher.to(device)
teacher.load_state_dict(torch.load('./teacher.pth'))

test_stats = evaluate(cifar10_test_loader, teacher, criterion, device)
print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

# Train the student
for p in teacher.parameters():
    p.requires_grad = False

MODEL_NAME = 'vit_tiny_patch16_224'

model = create_model(
        MODEL_NAME,
        pretrained=True,
        num_classes=10,
        img_size=224)
device = 'cuda:0' # device = 'cpu'
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WD)


n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('number of params:', n_parameters)


print(f"Start training for {EPOCHS} epochs")

for epoch in range(1, EPOCHS+1):
    train_stats = train_one_epoch_distillation(
        teacher, model, criterion, cifar10_training_loader,
        optimizer, device, epoch, alpha=2.0, temp=1.0)
    if epoch % 2 == 1:
        test_stats = evaluate(cifar10_test_loader, model, criterion, device)
        print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

test_stats = evaluate(cifar10_test_loader, model, criterion, device)
print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")